```

Sip = [S_x; S_y; S_z];
TCP = [X; Y; Z];

Rx = [1, 0, 0; 0, cos(alpha), sin(alpha); 0, -sin(alpha), cos(alpha)];
Ry = [cos(beta), 0, -sin(beta); 0, 1, 0; sin(beta), 0, cos(beta)];
Rz = [cos(gama), sin(gama), 0; -sin(gama), cos(gama), 0; 0, 0, 1];
R = Rx*Ry*Rz;

v = [v1; v2; v3];
x = [X; Y; Z]-L*R*v;
o = R*v;


a = sqrt(((-R(1,:)*v*TCP(3))/(R(3,:)*v))^2+(-(R(2,:)*v)*TCP(3)/(R(3,:)*v))^2+(-TCP(3))^2);
Q = a*o+TCP;
Sip_proj = Sip+(o'*Q-o'*Sip)*(x-Sip)/(o'*x-o'*Sip);
s_2 = [n1; n2; n3];
s_1 = cross(o,s_2);
k_1 = ((Sip_proj-Q)*s_1)/norm(s_1)^2;
k_2 = ((Sip_proj-Q)*s_2)/norm(s_2)^2;
Q_t = [0; 0];

% Obraz v zobrazovaci rovine
f_1 = (Q_t(1)+k_1*ff/(a+L-ff));
f_2 = Q_t(1)+k_2*ff/(a+L-ff);

% Derivace:
df1_X = diff(f_1, X);
df1_Y = diff(f_1, Y);
df1_Z = diff(f_1, Z);
df1_alpha = diff(f_1, alpha);
df1_beta = diff(f_1, beta);
df1_gama = diff(f_1, gama);
 
df2_X = diff(f_2, X);
df2_Y = diff(f_2, Y);
df2_Z = diff(f_2, Z);
df2_alpha = diff(f_2, alpha);
df2_beta = diff(f_2, beta);
df2_gama = diff(f_2, gama);

```

In [8]:
import sympy as sp

# Define the symbols
alpha, beta, gama, X, Y, Z, L, v1, v2, v3, n1, n2, n3, ff, S_x, S_y, S_z = sp.symbols(
    'alpha beta gama X Y Z L v1 v2 v3 n1 n2 n3 ff S_x S_y S_z', real=True)

# Sip and TCP
Sip = sp.Matrix([S_x, S_y, S_z])
TCP = sp.Matrix([X, Y, Z])

# Rotation matrices Rx, Ry, Rz
Rx = sp.Matrix([[1, 0, 0], [0, sp.cos(alpha), sp.sin(alpha)], [0, -sp.sin(alpha), sp.cos(alpha)]])
Ry = sp.Matrix([[sp.cos(beta), 0, -sp.sin(beta)], [0, 1, 0], [sp.sin(beta), 0, sp.cos(beta)]])
Rz = sp.Matrix([[sp.cos(gama), sp.sin(gama), 0], [-sp.sin(gama), sp.cos(gama), 0], [0, 0, 1]])
R = sp.simplify(Rx * Ry * Rz)

# Vector v and calculations for x and o
v = sp.Matrix([v1, v2, v3])
x = sp.Matrix([X, Y, Z]) - L * R * v
o = R * v

# Calculate 'a' using dot products
a = sp.simplify(sp.sqrt(((-R.row(0).dot(v) * Z) / (R.row(2).dot(v)))**2 +
                        (-(R.row(1).dot(v) * Z) / (R.row(2).dot(v)))**2 + Z * Z))

# Calculate Q
Q = a * o + TCP

# Project Sip onto the line defined by o and Q
Sip_proj = Sip + (o.dot(Q) - o.dot(Sip)) * (x - Sip) / (o.dot(x) - o.dot(Sip))

# s_2 vector and s_1 as the cross product of o and s_2
s_2 = sp.Matrix([n1, n2, n3])
s_1 = o.cross(s_2)

# Calculate k_1 and k_2
k_1 = (Sip_proj - Q).dot(s_1) / s_1.norm()**2
k_2 = (Sip_proj - Q).dot(s_2) / s_2.norm()**2

# Q_t as a zero vector (for 2D image plane coordinates)
Q_t = sp.Matrix([0, 0])

# Calculate image plane coordinates f_1 and f_2
f_1 = Q_t[0] + k_1 * ff / (a + L - ff)
f_2 = Q_t[1] + k_2 * ff / (a + L - ff)
f_both = sp.Matrix([f_1, f_2])

In [9]:
import inspect

args = [alpha, beta, gama, X, Y, Z, L, v1, v2, v3, n1, n2, n3, ff, S_x, S_y, S_z]
f_lambd = sp.lambdify(args, f_both, modules='numpy', cse=True)

print(inspect.getsource(f_lambd))

def _lambdifygenerated(alpha, beta, gama, X, Y, Z, L, v1, v2, v3, n1, n2, n3, ff, S_x, S_y, S_z):
    x0 = sin(beta)
    x1 = cos(beta)
    x2 = cos(gama)
    x3 = v1*x1*x2
    x4 = sin(gama)
    x5 = v2*x1*x4
    x6 = -v3*x0 + x3 + x5
    x7 = sin(alpha)
    x8 = x2*x7
    x9 = cos(alpha)
    x10 = x4*x9
    x11 = -x0*x10 + x8
    x12 = -v2*x11
    x13 = x4*x7
    x14 = x2*x9
    x15 = v1*(x0*x14 + x13)
    x16 = v3*x1
    x17 = x16*x9
    x18 = x15 + x17
    x19 = x12 + x18
    x20 = n1*x19 - n3*x6
    x21 = x16*x7
    x22 = v2*(x0*x13 + x14)
    x23 = v1*(x0*x8 - x10)
    x24 = x21 + x22 + x23
    x25 = n1*x24 - n2*x6
    x26 = -v2*x11 + x18
    x27 = -n3*x24
    x28 = -X
    x29 = sqrt(v1**2 + v2**2 + v3**2)*abs(Z/x26)
    x30 = x29*x6
    x31 = -L*v3*x0 + L*x3 + L*x5 + x28
    x32 = -Z
    x33 = L*x12 + L*x15 + L*x17 + x32
    x34 = -Y
    x35 = L*x21 + L*x22 + L*x23 + x34
    x36 = S_x*x6 + S_y*x24 + S_z*x19
    x37 = x24*x29
    x38 = x19*x29
    x39 = (x19*(Z + x38) + x24*(Y + 

In [7]:

reduced_exprs, simplified_expr = sp.cse(f_both)
reduced_exprs

[(x0, sin(beta)),
 (x1, cos(beta)),
 (x2, cos(gama)),
 (x3, v1*x1*x2),
 (x4, sin(gama)),
 (x5, v2*x1*x4),
 (x6, -v3*x0 + x3 + x5),
 (x7, sin(alpha)),
 (x8, x2*x7),
 (x9, cos(alpha)),
 (x10, x4*x9),
 (x11, -x0*x10 + x8),
 (x12, -v2*x11),
 (x13, x4*x7),
 (x14, x2*x9),
 (x15, v1*(x0*x14 + x13)),
 (x16, v3*x1),
 (x17, x16*x9),
 (x18, x15 + x17),
 (x19, x12 + x18),
 (x20, n1*x19 - n3*x6),
 (x21, x16*x7),
 (x22, v2*(x0*x13 + x14)),
 (x23, v1*(x0*x8 - x10)),
 (x24, x21 + x22 + x23),
 (x25, n1*x24 - n2*x6),
 (x26, -v2*x11 + x18),
 (x27, -n3*x24),
 (x28, -X),
 (x29, sqrt(v1**2 + v2**2 + v3**2)*Abs(Z/x26)),
 (x30, x29*x6),
 (x31, -L*v3*x0 + L*x3 + L*x5 + x28),
 (x32, -Z),
 (x33, L*x12 + L*x15 + L*x17 + x32),
 (x34, -Y),
 (x35, L*x21 + L*x22 + L*x23 + x34),
 (x36, S_x*x6 + S_y*x24 + S_z*x19),
 (x37, x24*x29),
 (x38, x19*x29),
 (x39,
  (x19*(Z + x38) + x24*(Y + x37) - x36 + x6*(X + x30))/(-x19*x33 - x24*x35 - x31*x6 - x36)),
 (x40, S_x + x28 - x30 + x39*(-S_x - x31)),
 (x41, S_y + x34 - x37 + x39*

In [21]:
from sympy import symbols, cse, lambdify


args = sp.Matrix([X, Y, Z, alpha, beta, gama])
# Now we generate the function code


def generate_function_code(reduced_exprs, simplified_expr, args):
    # Start the function definition
    func_code = "def compute_expression({}):".format(', '.join(str(arg) for arg in args))

    # Add lines to define intermediate variables
    for var, subexpr in reduced_exprs:
        func_code += "\n    {} = {}".format(var, subexpr)

    # Add line for the final result using the simplified expression
    func_code += "\n    return {}".format(simplified_expr[0])

    return func_code



# Generate the function code
func_code = generate_function_code(reduced_exprs, simplified_expr, args)

# Display the generated function code
print(func_code)

def compute_expression(X, Y, Z, alpha, beta, gama):
    x0 = sin(alpha)
    x1 = cos(gama)
    x2 = x0*x1
    x3 = sin(beta)
    x4 = sin(gama)
    x5 = cos(alpha)
    x6 = x4*x5
    x7 = x2 - x3*x6
    x8 = x0*x4
    x9 = x1*x5
    x10 = v1*(x3*x9 + x8)
    x11 = cos(beta)
    x12 = v3*x11
    x13 = x12*x5
    x14 = x10 + x13
    x15 = -v2*x7 + x14
    x16 = sqrt(Z**2*(v1**2 + v2**2 + v3**2)/x15**2)
    x17 = v1*x1*x11
    x18 = v2*x11*x4
    x19 = -v3*x3 + x17 + x18
    x20 = -v2*x7
    x21 = x14 + x20
    x22 = n1*x21 - n3*x19
    x23 = x0*x12
    x24 = v2*(x3*x8 + x9)
    x25 = v1*(x2*x3 - x6)
    x26 = x23 + x24 + x25
    x27 = n1*x26 - n2*x19
    x28 = -n3*x26
    x29 = -X
    x30 = x16*x19
    x31 = -L*v3*x3 + L*x17 + L*x18 + x29
    x32 = -Z
    x33 = L*x10 + L*x13 + L*x20 + x32
    x34 = -Y
    x35 = L*x23 + L*x24 + L*x25 + x34
    x36 = S_x*x19 + S_y*x26 + S_z*x21
    x37 = x16*x26
    x38 = x16*x21
    x39 = (x19*(X + x30) + x21*(Z + x38) + x26*(Y + x37) - x36)/(-x19*x31 - x

In [104]:
f_2 = Q_t[1] + k_2 * ff / (a + L - ff)
print(f_2)


ff*(n1*(S_x - X - sqrt(v1**2 + v2**2 + v3**2)*(v1*cos(beta)*cos(gama) + v2*sin(gama)*cos(beta) - v3*sin(beta))*Abs(Z/(v1*(sin(alpha)*sin(gama) + sin(beta)*cos(alpha)*cos(gama)) - v2*(sin(alpha)*cos(gama) - sin(beta)*sin(gama)*cos(alpha)) + v3*cos(alpha)*cos(beta))) + (-L*v1*cos(beta)*cos(gama) - L*v2*sin(gama)*cos(beta) + L*v3*sin(beta) - S_x + X)*(-S_x*(v1*cos(beta)*cos(gama) + v2*sin(gama)*cos(beta) - v3*sin(beta)) - S_y*(v1*(sin(alpha)*sin(beta)*cos(gama) - sin(gama)*cos(alpha)) + v2*(sin(alpha)*sin(beta)*sin(gama) + cos(alpha)*cos(gama)) + v3*sin(alpha)*cos(beta)) - S_z*(v1*(sin(alpha)*sin(gama) + sin(beta)*cos(alpha)*cos(gama)) + v2*(-sin(alpha)*cos(gama) + sin(beta)*sin(gama)*cos(alpha)) + v3*cos(alpha)*cos(beta)) + (X + sqrt(v1**2 + v2**2 + v3**2)*(v1*cos(beta)*cos(gama) + v2*sin(gama)*cos(beta) - v3*sin(beta))*Abs(Z/(v1*(sin(alpha)*sin(gama) + sin(beta)*cos(alpha)*cos(gama)) - v2*(sin(alpha)*cos(gama) - sin(beta)*sin(gama)*cos(alpha)) + v3*cos(alpha)*cos(beta))))*(v1*cos(beta)*

In [106]:
# Define the vector of symbols
v = sp.Matrix([X, Y, Z, alpha, beta, gama])

# Compute the gradient of f_1 with respect to the vector v
gradient_f1 = sp.derive_by_array(f_1, v)


In [54]:
reduced_exprs, simplified_expr = sp.cse(gradient_f1)
subs_dict = {}
for var, subexpr in reduced_exprs:
    if subexpr.func == sp.Symbol:
        subs_dict[var] = subexpr
    else:
        subs_dict[var] = subexpr
subs_dict

{x0: sin(beta),
 x1: v3*x0,
 x2: cos(beta),
 x3: cos(gama),
 x4: v1*x3,
 x5: x2*x4,
 x6: sin(gama),
 x7: v2*x6,
 x8: x2*x7,
 x9: -x1 + x5 + x8,
 x10: sin(alpha),
 x11: x10*x3,
 x12: cos(alpha),
 x13: x12*x6,
 x14: x0*x13,
 x15: x11 - x14,
 x16: -v2*x15,
 x17: x10*x6,
 x18: x12*x3,
 x19: x0*x18,
 x20: x17 + x19,
 x21: v1*x20,
 x22: v3*x2,
 x23: x12*x22,
 x24: x21 + x23,
 x25: x16 + x24,
 x26: n1*x25,
 x27: -n3*x9 + x26,
 x28: -Y,
 x29: x10*x22,
 x30: L*x29,
 x31: x0*x17,
 x32: x18 + x31,
 x33: v2*x32,
 x34: x0*x11,
 x35: -x13 + x34,
 x36: v1*x35,
 x37: L*x33 + L*x36 + x28 + x30,
 x38: -S_y - x37,
 x39: -Z,
 x40: L*x16 + L*x21 + L*x23,
 x41: x39 + x40,
 x42: -x41,
 x43: x29 + x33 + x36,
 x44: -x37,
 x45: -X,
 x46: -L*v3*x0 + L*x5 + L*x8 + x45,
 x47: -x46,
 x48: S_x*x9 + S_y*x43 + S_z*x25,
 x49: x25*x42 + x43*x44 + x47*x9 - x48,
 x50: 1/x49,
 x51: x50*x9,
 x52: -v2*x15 + x24,
 x53: sqrt(Z**2*(v1**2 + v2**2 + v3**2)/x52**2),
 x54: x53*x9,
 x55: X + x54,
 x56: x43*x53,
 x57: Y + x56,
 x58: 

In [114]:

import sympy as sp


tmp = reduced_exprs[35][1]

octave_code = sp.printing.octave_code(simplified_expr)
print(octave_code)


{ff.*(x22.*(S_y + x34 - x37 + x39.*(-S_y - x35)) - x27.*(S_z + x32 - x38 + x39.*(-S_z - x33)) + (-n2.*x21 - x28).*(S_x + x29 - x30 + x39.*(-S_x - x31)))./((L - ff + x16).*(x22.^2 + x27.^2 + (n2.*x15 + x28).^2))}


In [108]:

gradient_f1_lambd = sp.lambdify(v, gradient_f1, modules='sympy',cse=True)
print(inspect.getsource(gradient_f1_lambd))

def _lambdifygenerated(X, Y, Z, alpha, beta, gama):
    x0 = sin(beta)
    x1 = v3*x0
    x2 = cos(beta)
    x3 = cos(gama)
    x4 = v1*x3
    x5 = x2*x4
    x6 = sin(gama)
    x7 = v2*x6
    x8 = x2*x7
    x9 = -x1 + x5 + x8
    x10 = sin(alpha)
    x11 = x10*x3
    x12 = cos(alpha)
    x13 = x12*x6
    x14 = -x0*x13 + x11
    x15 = -v2*x14
    x16 = x10*x6
    x17 = x12*x3
    x18 = x0*x17 + x16
    x19 = v1*x18
    x20 = v3*x2
    x21 = x12*x20
    x22 = x19 + x21
    x23 = x15 + x22
    x24 = n1*x23
    x25 = -n3*x9 + x24
    x26 = -Y
    x27 = x10*x20
    x28 = L*x27
    x29 = x0*x16 + x17
    x30 = v2*x29
    x31 = x0*x11 - x13
    x32 = v1*x31
    x33 = L*x30 + L*x32 + x26 + x28
    x34 = -S_y - x33
    x35 = -Z
    x36 = L*x15 + L*x19 + L*x21
    x37 = x35 + x36
    x38 = -x37
    x39 = x27 + x30
    x40 = x32 + x39
    x41 = -x33
    x42 = -X
    x43 = -L*v3*x0 + L*x5 + L*x8 + x42
    x44 = -x43
    x45 = S_x*x9 + S_y*x40 + S_z*x23
    x46 = x23*x38 + x40*x41 + x44*x9 - x45
  

In [95]:
subs_dict = {}
for var, subexpr in reduced_exprs:
    if subexpr.func == sp.Symbol:
        subs_dict[var] = subexpr
    else:
        subs_dict[var] = subexpr
subs_dict

{x0: sin(beta),
 x1: v3*x0,
 x2: cos(beta),
 x3: cos(gama),
 x4: v1*x2*x3,
 x5: sin(gama),
 x6: v2*x2*x5,
 x7: -x1 + x4 + x6,
 x8: sin(alpha),
 x9: x3*x8,
 x10: cos(alpha),
 x11: x10*x5,
 x12: x0*x11,
 x13: -x12 + x9,
 x14: -v2*x13,
 x15: x5*x8,
 x16: x10*x3,
 x17: x0*x16,
 x18: v1*(x15 + x17),
 x19: v3*x2,
 x20: x10*x19,
 x21: x18 + x20,
 x22: x14 + x21,
 x23: n1*x22,
 x24: -n3*x7 + x23,
 x25: Abs(x24),
 x26: x19*x8,
 x27: x0*x15,
 x28: x16 + x27,
 x29: v2*x28,
 x30: x0*x9,
 x31: -x11 + x30,
 x32: v1*x31,
 x33: x26 + x29 + x32,
 x34: n1*x33 - n2*x7,
 x35: Abs(x34),
 x36: -v2*x13 + x21,
 x37: -n3*x33,
 x38: n2*x36 + x37,
 x39: Abs(x38),
 x40: x25**2 + x35**2 + x39**2,
 x41: 1/x40,
 x42: -n2*x22 - x37,
 x43: -X,
 x44: sqrt(Z**2*(v1**2 + v2**2 + v3**2)/x36**2),
 x45: x44*x7,
 x46: -L*v3*x0 + L*x4 + L*x6 + x43,
 x47: -S_x - x46,
 x48: -Z,
 x49: L*x14 + L*x18 + L*x20,
 x50: x48 + x49,
 x51: -x50,
 x52: -Y,
 x53: L*x26,
 x54: L*x29 + L*x32 + x52 + x53,
 x55: -x54,
 x56: S_x*x7 + S_y*x33 + S

In [100]:
reduced_exprs, simplified_expr = sp.cse(gradient_f1[3])
reduced_exprs[140][1].subs(subs_dict).subs(subs_dict).subs(subs_dict).subs(
    subs_dict).subs(subs_dict).subs(subs_dict).subs(subs_dict)

(-Derivative(re(sin(gama)*cos(alpha)), alpha) + Derivative(re(sin(alpha)*sin(beta)*cos(gama)), alpha))*re(v1) + (Derivative(re(cos(alpha)*cos(gama)), alpha) + Derivative(re(sin(alpha)*sin(beta)*sin(gama)), alpha))*re(v2) - (-Derivative(im(sin(gama)*cos(alpha)), alpha) + Derivative(im(sin(alpha)*sin(beta)*cos(gama)), alpha))*im(v1) - (Derivative(im(cos(alpha)*cos(gama)), alpha) + Derivative(im(sin(alpha)*sin(beta)*sin(gama)), alpha))*im(v2) + Derivative(re(v3*sin(alpha)*cos(beta)), alpha)

In [72]:
import numpy as np
import inspect
f1_lambd = sp.lambdify(v, f_1, modules=["mpmath"],cse=True)

print(inspect.getsource(f1_lambd))


def _lambdifygenerated(X, Y, Z, alpha, beta, gama):
    x0 = sin(alpha)
    x1 = cos(gama)
    x2 = x0*x1
    x3 = sin(beta)
    x4 = sin(gama)
    x5 = cos(alpha)
    x6 = x4*x5
    x7 = x2 - x3*x6
    x8 = x0*x4
    x9 = x1*x5
    x10 = v1*(x3*x9 + x8)
    x11 = cos(beta)
    x12 = v3*x11
    x13 = x12*x5
    x14 = x10 + x13
    x15 = -v2*x7 + x14
    x16 = sqrt(Z**2*(v1**2 + v2**2 + v3**2)/x15**2)
    x17 = v1*x1*x11
    x18 = v2*x11*x4
    x19 = -v3*x3 + x17 + x18
    x20 = -v2*x7
    x21 = x14 + x20
    x22 = n1*x21 - n3*x19
    x23 = x0*x12
    x24 = v2*(x3*x8 + x9)
    x25 = v1*(x2*x3 - x6)
    x26 = x23 + x24 + x25
    x27 = n1*x26 - n2*x19
    x28 = -n3*x26
    x29 = -X
    x30 = x16*x19
    x31 = -L*v3*x3 + L*x17 + L*x18 + x29
    x32 = -Z
    x33 = L*x10 + L*x13 + L*x20 + x32
    x34 = -Y
    x35 = L*x23 + L*x24 + L*x25 + x34
    x36 = S_x*x19 + S_y*x26 + S_z*x21
    x37 = x16*x26
    x38 = x16*x21
    x39 = (x19*(X + x30) + x21*(Z + x38) + x26*(Y + x37) - x36)/(-x19*x31 - x

In [ ]:
df1_X = sp.diff(f_1, X)
df1_Y = sp.diff(f_1, Y)
df1_Z = sp.diff(f_1, Z)
df1_alpha = sp.diff(f_1, alpha)
df1_beta = sp.diff(f_1, beta)
df1_gama = sp.diff(f_1, gama)

df2_X = sp.diff(f_2, X)
df2_Y = sp.diff(f_2, Y)
df2_Z = sp.diff(f_2, Z)
df2_alpha = sp.diff(f_2, alpha)
df2_beta = sp.diff(f_2, beta)
df2_gama = sp.diff(f_2, gama)